note that this notebook can be viewed as a slideshow triggered by: File -> Download as -> Reveal.js slides (.html)

## PySDM tutorial: Intro
### authors: see [github.com/atmos-cloud-sim-uj](https://github.com/atmos-cloud-sim-uj/PySDM/graphs/contributors)
### copyright: Jagiellonian University
#### tutorial licence: CC-BY

### *PySDM* : particle-based simulations of atmospheric cloud microphysics

The package features a Pythonic implementation of the Super-Droplet Method (SDM) Monte-Carlo algorithm, hence the name.

PySDM simulates the dynamics of population of particles immersed in moist air. Depending on their size, the simulated particles are referred to as:
- atmospheric aerosol (sub-micrometre particles, acting as nuclei for condensation of water vapour),
- cloud droplets (with radius in the 1 to 100 micrometre range, with negligible fall speed), 
- rain drops (larger precipitating particles formed by collisions).

### The particle-based ansatz in a nutshell:
- the particles are modelled as point-like objects called **super-droplets**   
 (with each super-droplet representing a multiplicity $n$ of its real-world counterparts with identical properties such as radius $r$, nucleus radius $r_d$, ...);

- the **ambient air** is modelled as a continuum medium - perfect gas mixture of dry air and water vapour (characterised by temperature $T$, total and partial pressures $p=p_d+p_v$, relative humidity $RH=p_v/p_{\text{sat}}(T)$ and density $\rho=\rho_d+\rho_v$, with $d$ and $v$ denoting dry-air and water-vapour, respectively);

- there is **bidirectional coupling** between the ambient air and the particles   
  (in particular: moisture present in the air drives particle growth by condensation, concurrent latent heat release alters the air temperature).

### The following model state description is embraced:
- moist air thermodynamic variables (see [Constants](constants.ipynb) for definition of $p_{1000}$, $R_d$, $c_{pd}$): 
  - dry-air density: $\rho_d$,
  - dry-air potential temperature: $\theta_d=T/\Pi=T\left(\frac{p_{1000}}{p_d}\right)^{\frac{R_d}{c_{pd}}}$, 
  - water vapour mixing ratio: $q_v=\frac{\rho_v}{\rho_d}$,
    
- attributes of $i$-th super-droplet:
  - multiplicity: $n^{[i]}$,
  - spatial coordinates (x,y,z), 
  - extensive attributes: particle volume $v^{[i]}$, nucleus volume, ...,
  - intensive attributes: ...

### Particle dynamics: collisional growth

Collisional growth is commonly modelled by the **Smoluchowski's coagulation equation** (formulated as continuum integro-differential equation in [Smoluchowski 1916](references.ipynb) under the assumptions of sufficiently large system and of neglected correlations between numbers of droplets of different sizes):

\begin{eqnarray}
\frac{d}{dt}{c_i} = \frac{1}{2} \sum\limits_{j=1}^{i-1} a_{j, i-j} c_j c_{i-j} - \sum\limits_{j=1}^{\infty} a_{j, i} c_j c_i
\end{eqnarray}

where $a_{i,j}$ is a so-called kernel defining rate of collisions and $c_i$ is the number concentration (in a volume of air) of the particles with volume $i \cdot \Delta v$ where $\Delta v$ is a droplet volume increment.

Note that $c_i$ will correspond to super-droplet multiplicity $n^{[i]}$ (in a unit volume of air) if:
- $v^{[i]} = i \cdot \Delta v$,
- $v^{[i]}$ does not change in time.

In order to relax the above conditions, a congruent stochastic formulation solved with a Monte-Carlo algorithm is used in PySDM  (see [Coalescence](coalescence.ipynb)).

### Particle dynamics: diffusional growth

Diffusional growth is modelled with the **Fick's** law depicting the proportionality of the mass flux of vapour per surface of droplet of radius $r$ to the gradient of the vapour density (ambient $\rho_v=q_v \rho_d$ vs. phase equilibrium at particle surface $\rho_\circ$):

\begin{eqnarray}
\frac{\dot{m}}{4 \pi r^2} =  
  D \frac{q_\text{v}\rho_d - \rho_\circ\left(r, \rho_d, \theta_d, q_v\right)}{r}
\end{eqnarray}

Particle growth dynamics resulting from the radius dependence in $\rho_\circ$ have the following phase portrait $\dot{\xi}(\xi)$, where $\xi=r^2$ and $RH$ is the relative humidity:

\begin{eqnarray}
  \dot{\xi} = (RH - 1) - \frac{a}{r} + \frac{b}{r^3}
\end{eqnarray}

![Arabas_and_Shima_2017_Fig1](pics/Arabas_and_Shima_2017_Fig1.svg)

### Particles-ambient air coupling

Two simplest frameworks: adiabatic parcel model and a kinematic flow:

$\begin{eqnarray}
\text{Lagrangian (adiabatic):}\\
\color{red}{\partial_t \rho_d}=\dot{\rho}_d(z) \\
\color{blue}{\partial_t q_v} = 0 \\
\color{green}{\partial_t \theta_d} = 0\\
\text{Eulerian (nondivergent):}\\
\color{red}{\partial_t\rho_d} = 0 = -\nabla\cdot(\vec{u}\rho_d) \\
\color{blue}{\partial_t q_v} = - \rho_d^{-1} \nabla \cdot (\vec{u} \rho_d q_v) \\
\color{green}{\partial_t \theta_d} = - \rho_d^{-1} \nabla \cdot (\vec{u} \rho_d \theta_d)
\end{eqnarray}$
$\qquad$
$\frac{d}{dt} \left[\begin{eqnarray} 
    v^{[i]} \\ 
    \vdots \\
    \rho_d \\
    \\
    \theta_d \\ 
    \\
    q_v
    \\
    ~
  \end{eqnarray} \right]$
  $=\left[\begin{eqnarray} 
    \dot{m}(v^{[i]}, \rho_d, \theta_d, q_v) / \rho_l\\
    \vdots \\ 
    \color{red}{\partial_t \rho_d} \\
    \\
    -\frac{l_v(T)}{c_p(q_v)} \frac{dq_v}{dt} \frac{1}{\Pi(\rho_d, \theta_d)} + \color{blue}{\partial_t \theta_d}  \\ 
     -\frac{\rho_l}{\rho_d} \frac{1}{\Delta V} \sum\limits_i n^{[i]} \frac{dv^{[i]}}{dt} + \color{green}{\partial_t q_v}
  \end{eqnarray}\right]$

where $l_v(T)$ is the latent heat of evaporation, $c_p(q_\text{v})$ is the specific heat of moist air.

In the Eulerian framework, particle positions evolve due sedimentation with terminal velocity $u_\text{term}$ and due to advection with the flow: $$[\dot{x}^{[i]}, \dot{y}^{[i]}, \dot{z}^{[i]}] = \vec{u} + \hat{e}_z u_\text{term}(v^{[i]})$$